# ◆[Question Classification（To find the intent of the Question）](https://www.kaggle.com/ananthu017/question-classification/kernels)
### 目的：あらゆる質問の文章の特徴から対応するカテゴリを予測する
- 説明変数が文章（TfidfVectorizerでベクトル化）で目的変数をカテゴリとするソフトマックス関数の多クラス分類モデルを構築する
###### Create Date：2020/01/08　Author：M.Hasegawa
### ────────────────────────────────────────────────────────────────

#### 【手順】
0. [**Notebook内で利用する関数の定義**](#import_module)
1. [**データ読込**](#import_data)
2. [**モデル構築と評価**](#modeling)

###### ※データセット「Question_Classification_Dataset.csv」（データ数：5,452行5列）

|No|項目|説明|型|特記事項|
|:---:|:---:|:---:|:---:|:---:|
|1|id|識別ID|数値||
|2|Questions|質問|カテゴり||
|3|Category0|カテゴリ0|カテゴリ|6種類の多クラス分類|
|4|Category1|カテゴリ1|カテゴリ|6種類の多クラス分類|
|5|Category2|カテゴリ2|カテゴリ|46種類の多クラス分類|

## 0. Notebook内で利用する関数の定義<a id='import_module'></a>

In [1]:
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import pickle
import os

import warnings
warnings.filterwarnings('ignore')

# 1. データ読込<a id='import_data'><a>

In [8]:
df= pd.read_csv('./Question_Classification_Dataset.csv')

print("\n▼ 行列数表示")
display(df.shape)

print("\n▼ 先頭5行表示")
display(df.head(5))

#print("\n▼ 基本統計量")
#display(df.describe())

#print('\n▼ 欠損データ確認')
#display(pd.DataFrame(df.isnull().sum()[df.isnull().sum()>0],columns=['Missing num']))

# 識別IDを除外
qn_df = df.iloc[:,1:]

print('\n▼ 各カテゴリの種類')
print('\nCategory0=',qn_df['Category0'].unique())
print('\nCategory1=',qn_df['Category1'].unique())
print('\nCategory2=',qn_df['Category2'].unique())


▼ 行列数表示


(5452, 5)


▼ 先頭5行表示


,Unnamed: 0,Questions,Category0,Category1,Category2
0,0,How did serfdom develop in and then leave Russ...,DESCRIPTION,DESC,manner
1,1,What films featured the character Popeye Doyle ?,ENTITY,ENTY,cremat
2,2,How can I find a list of celebrities ' real na...,DESCRIPTION,DESC,manner
3,3,What fowl grabs the spotlight after the Chines...,ENTITY,ENTY,animal
4,4,What is the full form of .com ?,ABBREVIATION,ABBR,exp



▼ 各カテゴリの種類

Category0= ['DESCRIPTION' 'ENTITY' 'ABBREVIATION' 'HUMAN' 'NUMERIC' 'LOCATION']

Category1= ['DESC' 'ENTY' 'ABBR' 'HUM' 'NUM' 'LOC']

Category2= ['manner' 'cremat' 'animal' 'exp' 'ind' 'gr' 'title' 'def' 'date' 'reason'
 'event' 'state' 'desc' 'count' 'other' 'letter' 'religion' 'food'
 'country' 'color' 'termeq' 'city' 'body' 'dismed' 'mount' 'money'
 'product' 'period' 'substance' 'sport' 'plant' 'techmeth' 'volsize'
 'instru' 'abb' 'speed' 'word' 'lang' 'perc' 'code' 'dist' 'temp' 'symbol'
 'ord' 'veh' 'weight' 'currency']


# 2. モデル構築と評価<a id='modeling'></a>
## 2-1.Category0の予測

In [14]:
# pd.factorizeでカテゴリ0を数値に変換
qn_df0 = qn_df[['Questions', 'Category0']]
qn_df0['Category0 Vectors'] = pd.factorize(qn_df0['Category0'])[0]

X = qn_df0['Questions']         # 説明変数（文字列の質問）
y = qn_df0['Category0 Vectors'] # 目的変数（数値化したカテゴリ0）

X_train0, X_test0, y_train0, y_test0 = train_test_split(X,y,test_size=0.2, random_state=1234)

# 説明変数の文章をベクトル化
vect = TfidfVectorizer(ngram_range = (1,2)).fit(qn_df0['Questions'])
X_train0_vector = vect.transform(X_train0)
X_test0_vector = vect.transform(X_test0)

#print('\nX_train=',X_train.shape)            # ベクトル化前
#print('\ntrain_vector=',train_vector.shape)  # ベクトル化後

# モデル生成、学習、予測、評価
model0 = SVC(kernel='linear', probability = True)
model0.fit(X_train0_vector, y_train0)
y_test0_pred = model0.predict(X_test0_vector)
print('カテゴリ0 正答率=',accuracy_score(y_test0_pred, y_test0))

# 閾値を適用
max_prob, max_prob_args = [],[]
prob = model0.predict_proba(X_test0_vector)
for i in range(len(prob)):
    max_prob.append(prob[i].max()) # 予測確率の最大値
    if prob[i].max() > 0.8:
        max_prob_args.append(prob[i].argmax()) # 80%より高い予測確率の場合、予測したカテゴリ0の数値を設定
    else:
        max_prob_args.append(-1) # うまく予測できなかった場合、-1を設定
a = pd.DataFrame(X_test0)
a['pred'] = max_prob_args # 予測したカテゴリ0（数値）※予測確率が低い場合は-1が設定されている
a['actual'] = y_test0     # 実際のカテゴリ0（数値）
a['max_prob'] = max_prob  # 予測確率
b = a[a['pred'] != -1]    # 予測が-1の行を除外する

print('カテゴリ0 正答率（※閾値指定）=',accuracy_score(b['pred'], b['actual']))

カテゴリ0 正答率= 0.8551787351054079
カテゴリ0 正答率（※閾値指定）= 0.9720744680851063


## 2-2.Category1の予測

In [16]:
# pd.factorizeでカテゴリ1を数値
qn_df1 = qn_df[['Questions', 'Category1']]
qn_df1['Category1 Vectors'] = pd.factorize(qn_df1['Category1'])[0]

X = qn_df1['Questions']         # 説明変数（文字列の質問）
y = qn_df1['Category1 Vectors']  # 目的変数（数値化したカテゴリ1）

X_train1, X_test1, y_train1, y_test1 = train_test_split(X,y, test_size=0.2, random_state=2345)

# 説明変数の文章をベクトル化
X_train1_vector = vect.transform(X_train1)
X_test1_vector = vect.transform(X_test1)

# モデル構築、学習、予測、評価
model1 = SVC(kernel='linear', probability = True)
model1.fit(X_train1_vector, y_train1)
y_test1_pred = model1.predict(X_test1_vector)
print('カテゴリ1 正答率=',accuracy_score(y_test1_pred, y_test1))

max_prob, max_prob_args = [],[]
prob = model1.predict_proba(X_test1_vector)
for i in range(len(prob)):
    max_prob.append(prob[i].max())
    if prob[i].max() > 0.8:
        max_prob_args.append(prob[i].argmax())
    else:
        max_prob_args.append(-1)
a = pd.DataFrame(X_test1)
a['pred'] = max_prob_args
a['actual'] = y_test1
a['max_prob'] = max_prob
b = a[a['pred'] != -1]

print('カテゴリ1 正答率（※閾値指定）=',accuracy_score(b['pred'], b['actual']))

カテゴリ1 正答率= 0.8386801099908341
カテゴリ1 正答率（※閾値指定）= 0.9712041884816754


## 2-3.Category2の予測

In [17]:
# pd.factorizeでカテゴリ2を数値に変換
qn_df2 = qn_df[['Questions', 'Category2']]
qn_df2['Category2 Vectors'] = pd.factorize(qn_df2['Category2'])[0]

X = qn_df2['Questions']         # 説明変数（文字列の質問）
y = qn_df2['Category2 Vectors']  # 目的変数（ベクトル化したカテゴリ2）

X_train2, X_test2, y_train2, y_test2 = train_test_split(X,y, test_size=0.2, random_state=3456)

# 説明変数をベクトル化
X_train2_vector = vect.transform(X_train2)
X_test2_vector = vect.transform(X_test2)

# モデル構築、学習、予測、評価
model2 = SVC(kernel='linear', probability = True)
model2.fit(X_train2_vector, y_train2)
y_test2_pred = model2.predict(X_test2_vector)
print('カテゴリ2 正答率=',accuracy_score(y_test2_pred, y_test2))

# 閾値を指定
max_prob, max_prob_args = [],[]
prob = model2.predict_proba(X_test2_vector)
for i in range(len(prob)):
    max_prob.append(prob[i].max())
    if prob[i].max() > 0.8:
        max_prob_args.append(prob[i].argmax())
    else:
        max_prob_args.append(-1)
a = pd.DataFrame(X_test2)
a['pred'] = max_prob_args
a['actual'] = y_test2
a['max_prob'] = max_prob
b = a[a['pred'] != -1]

print('カテゴリ2 正答率（※閾値指定）=',accuracy_score(b['pred'], b['actual']))

カテゴリ2 正答率= 0.7121906507791017
カテゴリ2 正答率（※閾値指定）= 0.9725158562367865


## Appendix.カテゴリと数値の辞書

In [6]:
dict_cat0 = {}
for val in qn_df0['Category0'].unique():
    dict_cat0[val] = qn_df0[qn_df0['Category0'] == val]['Category0 Vectors'].unique()[0]
print('\n▼dict_cat0=',dict_cat0)

dict_cat1 = {}
for val in qn_df1['Category1'].unique():
    dict_cat1[val] = qn_df1[qn_df1['Category1'] == val]['Category1 Vectors'].unique()[0]
print('\n▼dict_cat1=',dict_cat1)

dict_cat2 = {}
for val in qn_df2['Category2'].unique():
    dict_cat2[val] = qn_df2[qn_df2['Category2'] == val]['Category2 Vectors'].unique()[0]
print('\n▼dict_cat2=',dict_cat2)



▼dict_cat0= {'DESCRIPTION': 0, 'ENTITY': 1, 'ABBREVIATION': 2, 'HUMAN': 3, 'NUMERIC': 4, 'LOCATION': 5}

▼dict_cat1= {'DESC': 0, 'ENTY': 1, 'ABBR': 2, 'HUM': 3, 'NUM': 4, 'LOC': 5}

▼dict_cat2= {'manner': 0, 'cremat': 1, 'animal': 2, 'exp': 3, 'ind': 4, 'gr': 5, 'title': 6, 'def': 7, 'date': 8, 'reason': 9, 'event': 10, 'state': 11, 'desc': 12, 'count': 13, 'other': 14, 'letter': 15, 'religion': 16, 'food': 17, 'country': 18, 'color': 19, 'termeq': 20, 'city': 21, 'body': 22, 'dismed': 23, 'mount': 24, 'money': 25, 'product': 26, 'period': 27, 'substance': 28, 'sport': 29, 'plant': 30, 'techmeth': 31, 'volsize': 32, 'instru': 33, 'abb': 34, 'speed': 35, 'word': 36, 'lang': 37, 'perc': 38, 'code': 39, 'dist': 40, 'temp': 41, 'symbol': 42, 'ord': 43, 'veh': 44, 'weight': 45, 'currency': 46}
